# Day 6 - Tuning Trouble

https://adventofcode.com/2022/day/6


In [41]:
from pathlib import Path
import ipytest

ipytest.autoconfig(addopts=["-v"])  # type: ignore
INPUTS = Path("input.txt").read_text().strip()

## The short version

Truth be told I did this challenge overnight using a [Replit](https://replit.com/) because it was fairly straightforward:


In [42]:
def get_thing(num: int) -> int:
    for i in range(num, len(INPUTS)):
        if len(set(INPUTS[i - num : i])) == num:
            return i


print(f"Part 1 result: {get_thing(4)}")
print(f"Part 2 result: {get_thing(14)}")

Part 1 result: 1582
Part 2 result: 3588


## The long version

So, now I come to writing up this notebook for the solution. The above, while it works, is a bit dull.

Let's make this a more convoluted OOP solution instead, shall we? :)

### Classes

Let's start by writing out our class architecture.

In [43]:
from typing import Generator


class Signal:
    """Class containing a fairly convoluted possible solution for the day."""

    START_PACKET_MARKER_LENGTH: int = 4
    START_MESSAGE_MARKER_LENGTH: int = 14

    def __init__(self, content: str):
        self.content = content

    def windows(self, window_len: int) -> Generator:
        """Generator that returns sliding windows of length `window_len`
        from the contents.
        """
        for i in range(window_len, len(self.content)):
            yield self.content[i - window_len : i]

    def window_sets(self, window_len: int) -> Generator:
        """Generator returning sets from sliding windows of length `window_len`."""
        for window in self.windows(window_len=window_len):
            yield set(window)

    def _find_start_of_marker(self, marker_len: int) -> int:
        """Solve the problem for a marker of length `marker_len`,
        locating the first position where the marker ENDS in the content string.
        """
        for idx, window_set in enumerate(self.window_sets(window_len=marker_len)):
            if len(window_set) == marker_len:
                return idx + marker_len
        raise ValueError(
            f"Could not locate marker of length {marker_len} in code contents"
        )

    @property
    def packet_marker_pos(self) -> int:
        """Returns END of the packer marker (length"""
        return self._find_start_of_marker(marker_len=self.START_PACKET_MARKER_LENGTH)

    @property
    def message_marker_pos(self) -> int:
        """Returns END of the message marker"""
        return self._find_start_of_marker(marker_len=self.START_MESSAGE_MARKER_LENGTH)

### Testing

Now what good would all this be without proper testing?

In [44]:
%%ipytest

import pytest

@pytest.mark.parametrize("content, expected", [
    ("mjqjpqmgbljsphdztnvjfqwrcgsmlb", 7),
    ("bvwbjplbgvbhsrlpgdmjqwftvncz", 5),
    ("nppdvjthqldpwncqszvftbrmjlhg", 6),
    ("nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg", 10),
    ("zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw", 11),
])
def test_finding_packet_marker(content: str, expected: int):
    code = Signal(content)
    assert code.packet_marker_pos == expected



@pytest.mark.parametrize("content, expected", [
    ("mjqjpqmgbljsphdztnvjfqwrcgsmlb", 19),
    ("bvwbjplbgvbhsrlpgdmjqwftvncz", 23),
    ("nppdvjthqldpwncqszvftbrmjlhg", 23),
    ("nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg", 29),
    ("zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw", 26),
])
def test_finding_message_marker(content: str, expected: int):
    code = Signal(content)
    assert code.message_marker_pos == expected

======================================= test session starts ========================================
platform darwin -- Python 3.10.7, pytest-7.2.0, pluggy-1.0.0 -- /Users/garice/Library/Caches/pypoetry/virtualenvs/griceturrble-advent-of-code-8jQN35Cx-py3.10/bin/python
cachedir: .pytest_cache
rootdir: /Users/garice/dev/gits/personal/advent-of-code/2022/day06
collecting ... collected 10 items

t_274ff130844149cda5e06b8d8856cf8c.py::test_finding_packet_marker[mjqjpqmgbljsphdztnvjfqwrcgsmlb-7] PASSED [ 10%]
t_274ff130844149cda5e06b8d8856cf8c.py::test_finding_packet_marker[bvwbjplbgvbhsrlpgdmjqwftvncz-5] PASSED [ 20%]
t_274ff130844149cda5e06b8d8856cf8c.py::test_finding_packet_marker[nppdvjthqldpwncqszvftbrmjlhg-6] PASSED [ 30%]
t_274ff130844149cda5e06b8d8856cf8c.py::test_finding_packet_marker[nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg-10] PASSED [ 40%]
t_274ff130844149cda5e06b8d8856cf8c.py::test_finding_packet_marker[zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw-11] PASSED [ 50%]
t_274ff130844149cda5e06b8d8856c

### Execution

Tests are passing, whew! Now we can finally get some answers!

In [45]:
code = Signal(INPUTS)
print(f"Part 1 result: {code.packet_marker_pos}")
print(f"Part 2 result: {code.message_marker_pos}")

Part 1 result: 1582
Part 2 result: 3588


Welp, there you have it. Why settle for a 4-line solution when you can rewrite it into a 45-line class, eh?